<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Algorithmic Trading 

**Chapter 07 &mdash; Working with Real-Time Data and Sockets**

## Visualizing Streaming Data with Plotly

### The Basics

In [1]:
import zmq
from datetime import datetime
import plotly.graph_objects as go

In [2]:
symbol = 'SYMBOL'

In [3]:
fig = go.FigureWidget()
fig.add_scatter()
fig

FigureWidget({
    'data': [{'type': 'scatter', 'uid': '57663ff7-26e6-4784-8734-d435737372ea'}], 'layout': {'template': '...'}
})

In [4]:
context = zmq.Context()

In [5]:
socket = context.socket(zmq.SUB)

In [6]:
socket.connect('tcp://0.0.0.0:5555')

<SocketContext(connect='tcp://0.0.0.0:5555')>

In [7]:
socket.setsockopt_string(zmq.SUBSCRIBE, 'SYMBOL')

In [8]:
times = list()
prices = list()

In [9]:
for _ in range(50):
    msg = socket.recv_string()
    t = datetime.now()
    times.append(t)
    _, price = msg.split()
    prices.append(float(price))
    fig.data[0].x = times
    fig.data[0].y = prices

In [10]:
fig = go.FigureWidget()
fig.add_scatter(name='SYMBOL')
fig.add_scatter(name='SMA1', line=dict(width=1, dash='dot'),
                mode='lines+markers')
fig.add_scatter(name='SMA2', line=dict(width=1, dash='dash'),
                mode='lines+markers')
fig

FigureWidget({
    'data': [{'name': 'SYMBOL', 'type': 'scatter', 'uid': '89a602ed-4539-4801-bf00-ebe84493699b'},
             {'line': {'dash': 'dot', 'width': 1},
              'mode': 'lines+markers',
              'name': 'SMA1',
              'type': 'scatter',
              'uid': 'a2176055-36bf-46d7-9ec0-2d4eb17dad5c'},
             {'line': {'dash': 'dash', 'width': 1},
              'mode': 'lines+markers',
              'name': 'SMA2',
              'type': 'scatter',
              'uid': '11ba82f8-1d47-49d0-a47d-a927bb4c254a'}],
    'layout': {'template': '...'}
})

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame()

In [13]:
for _ in range(75):
    msg = socket.recv_string()
    t = datetime.now()
    sym, price = msg.split()
    # df = df.append(pd.DataFrame({sym: float(price)}, index=[t]))
    df = pd.concat([df, pd.DataFrame({sym: float(price)}, index=[t])])
    df['SMA1'] = df[sym].rolling(5).mean()
    df['SMA2'] = df[sym].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['SMA1']
    fig.data[2].y = df['SMA2']

In [14]:
from plotly.subplots import make_subplots

In [15]:
f = make_subplots(rows=3, cols=1, shared_xaxes=True)
f.append_trace(go.Scatter(name='SYMBOL'), row=1, col=1)
f.append_trace(go.Scatter(name='RETURN', line=dict(width=1, dash='dot'),
                mode='lines+markers', marker={'symbol': 'triangle-up'}),
                row=2, col=1)
f.append_trace(go.Scatter(name='MOMENTUM', line=dict(width=1, dash='dash'),
                mode='lines+markers', marker={'symbol': 'x'}), row=3, col=1)
# f.update_layout(height=600)

In [16]:
fig = go.FigureWidget(f)

In [17]:
fig

FigureWidget({
    'data': [{'name': 'SYMBOL',
              'type': 'scatter',
              'uid': '3b9dba63-9200-4fb3-9b9b-6e7ede531f72',
              'xaxis': 'x',
              'yaxis': 'y'},
             {'line': {'dash': 'dot', 'width': 1},
              'marker': {'symbol': 'triangle-up'},
              'mode': 'lines+markers',
              'name': 'RETURN',
              'type': 'scatter',
              'uid': '6fd845dd-ecdc-453e-8ae9-2a2cca0730c4',
              'xaxis': 'x2',
              'yaxis': 'y2'},
             {'line': {'dash': 'dash', 'width': 1},
              'marker': {'symbol': 'x'},
              'mode': 'lines+markers',
              'name': 'MOMENTUM',
              'type': 'scatter',
              'uid': 'e22ba656-caec-4fc8-b41b-0785748b6d97',
              'xaxis': 'x3',
              'yaxis': 'y3'}],
    'layout': {'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'matches': 'x3', 'showticklabels': False},
               'x

In [18]:
import numpy as np

In [19]:
df = pd.DataFrame()

In [20]:
for _ in range(75):
    msg = socket.recv_string()
    t = datetime.now()
    sym, price = msg.split()
    # df = df.append(pd.DataFrame({sym: float(price)}, index=[t]))
    df = pd.concat([df, pd.DataFrame({sym: float(price)}, index=[t])])
    df['RET'] = np.log(df[sym] / df[sym].shift(1))
    df['MOM'] = df['RET'].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['RET']
    fig.data[2].y = df['MOM']

In [21]:
socket = context.socket(zmq.SUB)

In [22]:
socket.connect('tcp://0.0.0.0:5556')

<SocketContext(connect='tcp://0.0.0.0:5556')>

In [23]:
socket.setsockopt_string(zmq.SUBSCRIBE, '')

In [24]:
for _ in range(5):
    msg = socket.recv_string()
    print(msg)

20.771 69.968 56.450 11.939 64.297 29.343 42.722 6.290
6.815 76.054 86.808 77.359 9.411 77.443 81.603 69.970
69.953 88.998 33.314 10.501 12.157 8.253 99.523 42.205
63.269 34.353 50.844 50.410 57.786 64.742 53.547 32.064
36.197 57.831 82.870 1.465 46.714 92.598 78.228 2.097


In [25]:
fig = go.FigureWidget()
fig.add_bar()
fig

FigureWidget({
    'data': [{'type': 'bar', 'uid': 'f3f31968-2c92-47a0-9b5e-4be4d58215df'}], 'layout': {'template': '...'}
})

In [26]:
x = list('abcdefgh')
fig.data[0].x = x
for _ in range(100):
    msg = socket.recv_string()
    y = msg.split()
    y = [float(n) for n in y]
    fig.data[0].y = y

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>